In [1]:
%load_ext autoreload

In [2]:
%autoreload
from tqdm.notebook import tqdm
import numpy as np
from diff3d import DLA_diff3d
from diff2d import DLA_diff2d

import seaborn as sns
from collections import Counter
import itertools
import matplotlib.pyplot as plt

sns.set()
sns.set_context("talk")

In [ ]:
def generate_tree_diff3d(PS):
    x, y, z = [40, 60, 40]
    eta = 1
    dla = DLA_diff3d(seed=[x//2, y - 1, z//2], x = x, y = y, z = z, eta=eta, w = 1, PS = PS)

    while not dla.converged:
        dla.update()

    for t in tqdm(range(150)):
        dla.growth(t + 1)

        while not dla.converged:
            dla.update()
    return dla

In [4]:
def generate_tree_diff2d(PS):
    x, y = [40, 80]
    eta = 1
    dla = DLA_diff2d(seed=[x//2, y - 1], x = x, y = y, eta=eta, w = 1, PS = PS)

    while not dla.converged:
        dla.update()

    for t in tqdm(range(150)):
        dla.growth(t + 1)

        while not dla.converged:
            dla.update()
    return dla

In [6]:
simulations = 3
PSs = [10, 20, 30, 40, 50]

generated_trees = dict()


In [7]:
for PS in PSs:
    generated_trees[PS] = []

In [8]:
for PS in PSs:
    for _ in range(simulations):
        # change model here either
        # generate_tree_diff3d
        # or 
        # generate_tree_diff2d
        generated_trees[PS].append(generate_tree_diff3d(PS))
        
generated_trees

KeyboardInterrupt: 

In [ ]:
# for PS in PSs:
#     plt.figure()
#     branch_orders = []
#     for tree in generated_trees[PS]:
#         branch_orders += tree.tree.get_branch_orders()
#     plt.hist(branch_orders, bins = max(branch_orders))
#     plt.title("{}".format(PS))

In [ ]:
fig, ax = plt.subplots(2, 3, figsize = (9, 6))
ax = ax.flatten()
counter  = 0
for PS in PSs:

    branch_orders = []
    for tree in generated_trees[PS]:
        branch_orders += tree.tree.get_branch_orders()
    
    
    c = Counter(branch_orders)
    c = sorted(c.items())
    x, y = zip(*c)
    ax[counter].hist(branch_orders, alpha = 0.5, bins = max(branch_orders))
    ax[counter].set_title("PS: {}".format(PS))
    ax[counter].set_xlabel("Branching order")
    ax[counter].set_ylabel("Occurences")
    counter += 1

fig.delaxes(ax[counter])
fig.subplots_adjust(wspace = 1, hspace = 1.5)
fig.suptitle("Branching orders as a function of the Pruning Span", y = 1.03)
plt.ylabel("Occurences")
plt.show()

In [ ]:
plt.figure()
for PS in PSs:

    branch_orders = []
    for tree in generated_trees[PS]:
        branch_orders += tree.tree.get_branch_orders()
    
    
    c = Counter(branch_orders)
    c = sorted(c.items())
    x, y = zip(*c)
    plt.plot(x, y, label='PS = {}'.format(PS))
        
plt.title("Occurence of the branching orders per pruning span")
plt.xlabel("Branch order")
plt.ylabel("Occurence")
plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
plt.show()

In [ ]:
means = []
errors = []
for PS in PSs:
    length_ratio = []

    for tree in generated_trees[PS]:
        length_ratio.append(tree.tree.get_lenghts_ratio())
        
    mean = np.mean(length_ratio)
    std = np.std(length_ratio)
    means.append(mean)
    errors.append(std)
    
    print("Length ratio = {} +- {}. For PS = {}".format(mean, std, PS))
    

In [ ]:
fig, ax = plt.subplots()
ax.bar(PSs, means, yerr=errors, width = 5, alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Length ratio')
ax.set_xlabel('Pruning span')
ax.set_xticks(PSs)
ax.yaxis.grid(True)
fig.suptitle("Length ratio as a function of the pruning span")
plt.show()


In [ ]:
means = []
errors = []
for PS in PSs:
    asym = []

    for tree in generated_trees[PS]:
        asym.append(tree.tree.get_asymmetry_index())
    
    mean = np.mean(asym)
    std = np.std(asym)
    means.append(mean)
    errors.append(std)
    print("Asymmetry index = {} +- {}. For PS = {}".format(mean, std, PS))
    

In [ ]:
fig, ax = plt.subplots()
ax.bar(PSs, means, yerr=errors, width = 5, alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Asymmetry index')
ax.set_xlabel('Pruning span')
ax.set_xticks(PSs)
ax.yaxis.grid(True)
fig.suptitle("Asymmetry index as a function of the pruning span")
plt.show()

In [ ]:
# beware for change in function name, leafs!=terminal segments
means = []
errors = []

for PS in PSs:
    leafs = []

    for tree in generated_trees[PS]:
        leafs.append(tree.tree.get_number_of_terminal_segments())
    
    mean = np.mean(leafs)
    std = np.std(leafs)
    means.append(mean)
    errors.append(std)
    
    print("Terminal leafs = {} +- {}. For PS = {}".format(mean, std, PS))



In [ ]:
fig, ax = plt.subplots()
ax.bar(PSs, means, yerr=errors, width = 5, alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Terminal leafs')
ax.set_xlabel('Pruning span')
ax.set_xticks(PSs)
ax.yaxis.grid(True)
fig.suptitle("Terminal leafs as a function of the pruning span")
plt.show()